# Boundary Value Problem


$$
u''= \sin(2\pi x), \quad u'(0) = 0, \quad u'(1) = 0.
$$
---

## 1. Check the consistency condition

For a Neumann boundary value problem:

$$
u''(x) = f(x), \quad u'(0) = \alpha, \quad u'(1) = \beta,
$$

a solution exists if and only if the **consistency condition** holds:

$$
\int_0^1 f(x)\, dx = u'(1) - u'(0) = \beta - \alpha.
$$

In our problem:

$$
\alpha = 0, \quad \beta = 0, \quad f(x) = \sin(2\pi x),
$$

so

$$
\int_0^1 \sin(2\pi x)\, dx 
= \left[ -\frac{\cos(2\pi x)}{2\pi} \right]_0^1
= -\frac{\cos(2\pi)}{2\pi} + \frac{\cos(0)}{2\pi} = 0.
$$

The consistency condition is satisfied. So a solution exists, unique up to an additive constant (since Neumann BCs only determine $u$ up to a constant).



## 2. Exact Solution of the BVP

Consider the boundary value problem:

$$
u''(x) = \sin(2\pi x), \quad u'(0) = 0, \quad u'(1) = 0.
$$



Integrating $u''(x) = \sin(2\pi x)$ once gives:

$$
u'(x) = \int \sin(2\pi x) \, dx = -\frac{\cos(2\pi x)}{2\pi} + C_1
$$

where $C_1$ is a constant.

---
At $x = 0$:

$$
u'(0) = -\frac{\cos(0)}{2\pi} + C_1 = -\frac{1}{2\pi} + C_1 = 0 \implies C_1 = \frac{1}{2\pi}
$$

At $x = 1$:

$$
u'(1) = -\frac{\cos(2\pi)}{2\pi} + C_1 = -\frac{1}{2\pi} + \frac{1}{2\pi} = 0
$$

Both conditions are satisfied.

Hence:

$$
u'(x) = -\frac{\cos(2\pi x)}{2\pi} + \frac{1}{2\pi}
$$

---
Integrating once more:

$$
u(x) = \int u'(x)\, dx = \int \left(-\frac{\cos(2\pi x)}{2\pi} + \frac{1}{2\pi}\right) dx
= -\frac{\sin(2\pi x)}{(2\pi)^2} + \frac{x}{2\pi} + C_2
$$

where $C_2$ is an arbitrary constant, because Neumann boundary conditions only determine $u(x)$ up to an additive constant.

---

### Final exact solution

$$
u(x) = -\frac{\sin(2\pi x)}{4\pi^2} + \frac{x}{2\pi} + C_2
$$

$C_2$ can be chosen arbitrarily. For numerical comparison, we can take $C_2 = 0$.



In [7]:
import numpy as np

# ================================================================
# Exact solution
# ================================================================
def u_exact(x):
    return -np.sin(2*np.pi*x)/(4*np.pi**2) + x/(2*np.pi)  # C2=0


# RHS f(x)
def f(x):
    return np.sin(2*np.pi*x)


# ================================================================
# Solve with 2nd-order finite differences (Neumann BC)
# ================================================================
def solve_neumann(N):
    h = 1/N
    x = np.linspace(0,1,N+1)

    A = np.zeros((N+1, N+1))
    b = f(x)

    # Interior points
    for i in range(1, N):
        A[i, i-1] = 1/h**2
        A[i, i]   = -2/h**2
        A[i, i+1] = 1/h**2

    # Neumann left: u'(0)=0 → (u1−u0)/h = 0
    A[0,0] = -1/h
    A[0,1] =  1/h
    b[0] = 0

    # Neumann right: u'(1)=0 → (uN−u_{N−1})/h = 0
    A[N,N]   =  1/h
    A[N,N-1] = -1/h
    b[N] = 0

    # Fix constant: set u(0)=0
    A[0,:] = 0
    A[0,0] = 1
    b[0] = 0

    # Solve linear system
    u = np.linalg.solve(A, b)
    return x, u


# ================================================================
# Error norms
# ================================================================
def error_norms(N):
    x, u = solve_neumann(N)
    u_ex = u_exact(x)
    err = np.abs(u - u_ex)
    L_inf = np.max(err)
    L2 = np.sqrt(np.sum(err**2) * (1/N))
    return L_inf, L2


# ================================================================
# Compute convergence table
# ================================================================
Ns = [20, 40, 80, 160, 320]
results = []

prev_inf = None
prev_L2 = None

for N in Ns:
    L_inf, L2 = error_norms(N)
    if prev_inf is None:
        rate_inf = np.nan
        rate_L2 = np.nan
    else:
        rate_inf = np.log(prev_inf / L_inf) / np.log(2)
        rate_L2  = np.log(prev_L2 / L2) / np.log(2)

    results.append([N, L_inf, rate_inf, L2, rate_L2])
    prev_inf, prev_L2 = L_inf, L2


# ================================================================
# table printing
# ================================================================
header = (
    f"{'N':>6} | "
    f"{'L_inf error':>15} | {'rate':>7} | "
    f"{'L2 error':>15} | {'rate':>7}"
)

line = "-" * len(header)

print("\nConvergence Table (2nd-order FD with Neumann BCs)")
print(line)
print(header)
print(line)

for N, Linf, rinf, L2, r2 in results:
    print(
        f"{N:6d} | "
        f"{Linf:15.8e} | {rinf:7.3f} | "
        f"{L2:15.8e} | {r2:7.3f}"
    )

print(line)



Convergence Table (2nd-order FD with Neumann BCs)
--------------------------------------------------------------
     N |     L_inf error |    rate |        L2 error |    rate
--------------------------------------------------------------
    20 |  1.31115523e-03 |     nan |  7.42963348e-04 |     nan
    40 |  3.27383890e-04 |   2.002 |  1.81698209e-04 |   2.032
    80 |  8.18207209e-05 |   2.000 |  4.49369926e-05 |   2.016
   160 |  2.04536029e-05 |   2.000 |  1.11743821e-05 |   2.008
   320 |  5.11330236e-06 |   2.000 |  2.78617901e-06 |   2.004
--------------------------------------------------------------
